In [ ]:
import numpy as np
from keras.models import load_model
from keras.models import Model
from keras.layers import Input, Average
from keras.utils import to_categorical
from keras.datasets import cifar10

# İki farklı modelin ağırlıklı ortalamasını alan Ensemble modelini oluştur
def create_ensemble_model(model1, model2):
    for layer in model1.layers:
        layer.trainable = False
    for layer in model2.layers:
        layer.trainable = False

    output1 = model1.layers[-2].output
    output2 = model2.layers[-2].output

    merged = Average()([output1, output2])
    ensemble_model = Model(inputs=[model1.input, model2.input], outputs=merged)

    return ensemble_model

# İki farklı modeli yükle
model1 = load_model('model1.h5')  # İlk modelin dosya adını ve yolunu belirtin
model2 = load_model('model2.h5')  # İkinci modelin dosya adını ve yolunu belirtin

# Ensemble modelini oluştur
ensemble_model = create_ensemble_model(model1, model2)

# Veriyi yükle (örneğin, CIFAR-10 veri kümesi)
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_test = x_test.astype('float32') / 255.0
y_test = to_categorical(y_test, num_classes=10)

# Modeli değerlendir
ensemble_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
ensemble_model.summary()
evaluation = ensemble_model.evaluate([x_test, x_test], y_test)
print(f"Ensemble Model Accuracy: {evaluation[1] * 100:.2f}%")
